# Scraping Stats for Individual Player

In [31]:
import pandas as pd
import pickle
import os
import re
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import gc
%matplotlib inline

This notebook is used to pull the individual game stats for any NFL player.

Simply change the URL below to the appropriate code associated with that player.

In [32]:
df = pd.read_csv('./Player_Stats.csv')

In [33]:
df

,Player,PPR,Day,G#,Week,Season,Date,Age,Team,Home,Opp,Result,FantPt,PPR.1,DKPt,FDPt,Pos,Draft,Player-additional
0,Davante Adams,41.60,Sun,1,1,2020,2020-09-13,27-264,GNB,@,MIN,W 43-34,27.60,41.60,44.6,34.6,WR,Feb-53,NaN
1,Josh Jacobs,35.90,Sun,1,1,2020,2020-09-13,22-215,LVR,@,CAR,W 34-30,31.90,35.90,35.9,33.9,RB,24-Jan,NaN
2,Calvin Ridley,33.90,Sun,1,1,2020,2020-09-13,25-268,ATL,NaN,SEA,L 25-38,24.90,33.90,36.9,29.4,WR,26-Jan,NaN
3,Russell Wilson,31.78,Sun,1,1,2020,2020-09-13,31-289,SEA,@,ATL,W 38-25,31.78,31.78,34.8,31.8,QB,Mar-75,NaN
4,Adam Thielen,31.00,Sun,1,1,2020,2020-09-13,30-022,MIN,NaN,GNB,L 34-43,25.00,31.00,34.0,28.0,WR,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11194,Mitchell Wilcox,1.30,Sun,2,2,2022,2022-09-18,25-315,CIN,@,DAL,L 17-20,0.30,1.30,1.3,0.8,TE,NaN,WilcMi01
11195,Stephen Anderson,1.20,Sun,2,2,2022,2022-09-18,29-231,ARI,@,LVR,W 29-23 (OT),0.20,1.20,1.2,0.7,TE,NaN,AndeSt01
11196,Bryan Edwards,1.20,Sun,2,2,2022,2022-09-18,23-309,ATL,@,LAR,L 27-31,0.20,1.20,1.2,0.7,WR,3-81,EdwaBr01
11197,Adam Prentice,1.20,Sun,2,2,2022,2022-09-18,25-245,NOR,NaN,TAM,L 10-20,0.20,1.20,1.2,0.7,FB,NaN,PrenAd00


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5843 entries, 0 to 5842
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Player             5843 non-null   object 
 1   PPR                5843 non-null   float64
 2   Day                5843 non-null   object 
 3   G#                 5843 non-null   int64  
 4   Week               5843 non-null   int64  
 5   Season             5843 non-null   int64  
 6   Date               5843 non-null   object 
 7   Age                5843 non-null   object 
 8   Team               5843 non-null   object 
 9   Unnamed: 9         2902 non-null   object 
 10  Opp                5843 non-null   object 
 11  Result             5843 non-null   object 
 12  FantPt             5843 non-null   float64
 13  PPR.1              5843 non-null   float64
 14  DKPt               5843 non-null   float64
 15  FDPt               5843 non-null   float64
 16  Pos.               5843 

In [11]:
#Filters to pull a specific player

#player_filter = df.Player = ''
# EX: player_filter = df[df.Player == 'Tom Brady']

In [12]:
#Replace the "/B/BradTo00" portion of the url with the Player-additional
url = 'https://www.pro-football-reference.com/players/B/BradTo00/gamelog/'
tb = pd.read_html(url)[0]
tb = tb.iloc[:, :-15]

# making a list of the dual-headers
col_list = [col for col in tb]
col_list2 = []

# only grabbing the sub-header 
for col in col_list:
    col_list2.append(col[1])

# renaming columns as sub-header   
tb.columns = col_list2

# splitting the df into two separate df's
# applying numeric filter to number
# then slamming them back together
clean_tb = pd.concat(
    [tb.iloc[:,:37], tb.iloc[:,37:].apply(pd.to_numeric, errors='coerce')],
    axis=1)

clean_tb = clean_tb[clean_tb["Rk"].str.contains("Rk")==False]
clean_tb.drop(clean_tb.tail(1).index,inplace=True)
clean_tb.columns.values[7] = 'Home'
clean_tb['Home'] = clean_tb['Home'].apply(lambda x: 0 if x == '@' else 1)

clean_tb.columns.values[19] = 'Sk_Yds'
clean_tb.columns.values[22] = 'Ru_Att'
clean_tb.columns.values[23] = 'Ru_Yds'
clean_tb.columns.values[24] = 'Ru_Y/A'
clean_tb.columns.values[25] = 'Ru_TB'
clean_tb.columns.values[26] = 'Re_Tgt'
clean_tb.columns.values[27] = 'Re_Rec'
clean_tb.columns.values[28] = 'Re_Yds'
clean_tb.columns.values[29] = 'Re_Y/R'
clean_tb.columns.values[30] = 'Re_TD'
clean_tb.columns.values[31] = 'Re_Ctch%'
clean_tb.columns.values[32] = 'Re_Y/Tgt'
clean_tb.columns.values[33] = 'Total_NP_TD'
clean_tb = clean_tb.drop(columns= ['Result', 'GS'])


# looking at it in all its glory
clean_tb

,Rk,Year,Date,G#,Week,Age,Tm,Home,Opp,Cmp,...,Re_Rec,Re_Yds,Re_Y/R,Re_TD,Re_Ctch%,Re_Y/Tgt,Total_NP_TD,Pts,Fmb,FL
0,1,2000,2000-09-03,1,1,NaN,NWE,1,TAM,Inactive,...,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive
1,2,2000,2000-09-11,2,2,NaN,NWE,0,NYJ,Inactive,...,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive
2,3,2000,2000-09-17,3,3,NaN,NWE,1,MIN,Inactive,...,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive
3,4,2000,2000-09-24,4,4,NaN,NWE,0,MIA,Inactive,...,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive
4,5,2000,2000-10-01,5,5,NaN,NWE,0,DEN,Inactive,...,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive,Inactive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,353,2021,2022-01-09,17,18,44.159,TAM,1,CAR,29,...,0,0,NaN,0,0.0%,NaN,0,0,0,0
375,354,2022,2022-09-11,1,1,45.039,TAM,0,DAL,18,...,0,0,NaN,0,0.0%,NaN,0,0,0,0
376,355,2022,2022-09-18,2,2,45.046,TAM,0,NOR,18,...,0,0,NaN,0,0.0%,NaN,0,0,1,1
377,356,2022,2022-09-25,3,3,45.053,TAM,1,GNB,31,...,0,0,NaN,0,0.0%,NaN,0,0,0,0


In [13]:
clean_tb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 357 entries, 0 to 378
Data columns (total 35 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Rk           357 non-null    object
 1   Year         357 non-null    object
 2   Date         357 non-null    object
 3   G#           357 non-null    object
 4   Week         357 non-null    object
 5   Age          322 non-null    object
 6   Tm           357 non-null    object
 7   Home         357 non-null    int64 
 8   Opp          357 non-null    object
 9   Cmp          357 non-null    object
 10  Att          357 non-null    object
 11  Cmp%         357 non-null    object
 12  Yds          357 non-null    object
 13  TD           357 non-null    object
 14  Int          357 non-null    object
 15  Rate         357 non-null    object
 16  Sk           357 non-null    object
 17  Sk_Yds       357 non-null    object
 18  Y/A          357 non-null    object
 19  AY/A         357 non-null    

We now have a DF containing only info about Tom Brady. 

We need to change our numerical features to floats.

In [14]:
col_list = clean_tb.columns

In [15]:
col_list

Index(['Rk', 'Year', 'Date', 'G#', 'Week', 'Age', 'Tm', 'Home', 'Opp', 'Cmp',
       'Att', 'Cmp%', 'Yds', 'TD', 'Int', 'Rate', 'Sk', 'Sk_Yds', 'Y/A',
       'AY/A', 'Ru_Att', 'Ru_Yds', 'Ru_Y/A', 'Ru_TB', 'Re_Tgt', 'Re_Rec',
       'Re_Yds', 'Re_Y/R', 'Re_TD', 'Re_Ctch%', 'Re_Y/Tgt', 'Total_NP_TD',
       'Pts', 'Fmb', 'FL'],
      dtype='object')

In [16]:
col_list = col_list.drop(['Date', 'Tm', 'Opp'])

In [17]:
clean_tb[col_list] = clean_tb[col_list].apply(pd.to_numeric, errors='coerce', axis=1)

In [18]:
clean_tb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 357 entries, 0 to 378
Data columns (total 35 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rk           357 non-null    float64
 1   Year         357 non-null    float64
 2   Date         357 non-null    object 
 3   G#           357 non-null    float64
 4   Week         357 non-null    float64
 5   Age          322 non-null    float64
 6   Tm           357 non-null    object 
 7   Home         357 non-null    float64
 8   Opp          357 non-null    object 
 9   Cmp          322 non-null    float64
 10  Att          322 non-null    float64
 11  Cmp%         322 non-null    float64
 12  Yds          322 non-null    float64
 13  TD           322 non-null    float64
 14  Int          322 non-null    float64
 15  Rate         322 non-null    float64
 16  Sk           322 non-null    float64
 17  Sk_Yds       322 non-null    float64
 18  Y/A          322 non-null    float64
 19  AY/A    

# Creating a Feature for Dome Stadiums 

In [19]:
Dome_Dict = {
    "ARI" : 1,
    "ATL" : 1,
    "DAL" : 1,
    "DET" : 1,
    "HOU" : 1,
    "IND" : 1,
    "LAC" : 1,
    "LAR" : 1,
    "LVR" : 1,
    "MIN" : 1,
    "NOR" : 1,
}

In [20]:
clean_tb['Dome_Home'] = clean_tb['Tm'].apply(lambda x: 1 if x in Dome_Dict.keys() else 0)
clean_tb['Dome_Away'] = clean_tb['Opp'].apply(lambda x: 1 if x in Dome_Dict.keys() else 0)

In [21]:
clean_tb['Dome'] = clean_tb['Dome_Home'] + clean_tb['Dome_Away']

In [22]:
clean_tb = clean_tb.drop(columns= ['Dome_Home', 'Dome_Away'])

# Need to convert players stats into our PPR target.

In [23]:
clean_tb['passing_ppr'] = ((clean_tb['Yds'] / 25)) + (clean_tb['TD'] * 4) + (clean_tb['Int'] * -2)
clean_tb['rushing_ppr'] = ((clean_tb['Ru_Yds'] + clean_tb['Re_Yds']) / 10)
clean_tb['td_ppr'] = (clean_tb['Total_NP_TD'] * 6)
clean_tb['receiving_ppr'] = (clean_tb['Re_Rec'] * 1)
clean_tb['fumble_ppr'] = (clean_tb['FL'] * -2)
#clean_tb['total'] = sum(clean_tb[-6:])
clean_tb['Total'] = clean_tb.iloc[:, -6:-1].sum(axis=1)


# 1 point per 25 passing yards
# 4 points per passing touchdown
# -2 points per interception
# 1 point per reception
# 1 point per 10 yards rushing/receiving
# 6 points per TD
# 2 points per two-point conversion
# -2 points per fumble lost


In [24]:
clean_tb.tail(10)

,Rk,Year,Date,G#,Week,Age,Tm,Home,Opp,Cmp,...,Pts,Fmb,FL,Dome,passing_ppr,rushing_ppr,td_ppr,receiving_ppr,fumble_ppr,Total
368,348.0,2021.0,2021-12-05,12.0,13.0,44.124,TAM,0.0,ATL,38.0,...,0.0,0.0,0.0,1,28.72,-0.1,0.0,0.0,-0.0,29.62
369,349.0,2021.0,2021-12-12,13.0,14.0,44.131,TAM,1.0,BUF,31.0,...,6.0,0.0,0.0,0,22.52,1.6,6.0,0.0,-0.0,30.12
370,350.0,2021.0,2021-12-19,14.0,15.0,44.138,TAM,1.0,NOR,26.0,...,0.0,1.0,1.0,1,6.56,0.2,0.0,0.0,-2.0,7.76
371,351.0,2021.0,2021-12-26,15.0,16.0,44.145,TAM,0.0,CAR,18.0,...,0.0,0.0,0.0,0,13.28,1.1,0.0,0.0,-0.0,14.38
372,352.0,2021.0,2022-01-02,16.0,17.0,44.152,TAM,0.0,NYJ,34.0,...,0.0,0.0,0.0,0,26.40,0.0,0.0,0.0,-0.0,26.40
373,353.0,2021.0,2022-01-09,17.0,18.0,44.159,TAM,1.0,CAR,29.0,...,0.0,0.0,0.0,0,25.04,0.0,0.0,0.0,-0.0,25.04
375,354.0,2022.0,2022-09-11,1.0,1.0,45.039,TAM,0.0,DAL,18.0,...,0.0,0.0,0.0,1,10.48,-0.1,0.0,0.0,-0.0,11.38
376,355.0,2022.0,2022-09-18,2.0,2.0,45.046,TAM,0.0,NOR,18.0,...,0.0,1.0,1.0,1,11.60,-0.2,0.0,0.0,-2.0,12.40
377,356.0,2022.0,2022-09-25,3.0,3.0,45.053,TAM,1.0,GNB,31.0,...,0.0,0.0,0.0,0,14.84,-0.1,0.0,0.0,-0.0,14.74
378,357.0,2022.0,2022-10-02,4.0,4.0,45.060,TAM,1.0,KAN,39.0,...,0.0,1.0,1.0,0,27.40,0.0,0.0,0.0,-2.0,27.40


Dropping these features to avoid multicollinearity later

In [ ]:
clean_tb = clean_tb.drop(columns= ['passing_ppr', 'rushing_ppr', 'td_ppr', 'receiving_ppr', 'fumble_ppr'])

# Defensive Rankings and Stats for the last 6 years

In [25]:
defenses = pd.read_csv('./Defenses.csv')
defenses

,Year,RANK,NAME,PTS,FPTS,SACKS,INT,TO,PTS.1,PLAYS,...,RUSH YDS,RUSH ATT,RUSH TD,TD,RZ ATT,RZ TD,RZ TD%,1D,3D%,4D%
0,2022,1,TAM,6,45,11,5,8,27,191,...,238,63,0,3,5,3,60.0,44,30.2,33.3
1,2022,2,DEN,2,28,9,1,4,36,160,...,244,56,0,3,4,1,25.0,47,26.5,25.0
2,2022,3,SFO,0,25,8,3,4,37,173,...,236,84,2,5,4,3,75.0,43,32.5,0.0
3,2022,4,BUF,8,41,11,5,7,38,158,...,173,62,3,5,6,5,83.3,46,39.4,75.0
4,2022,4,JAX,0,36,7,6,8,38,176,...,165,53,0,5,7,3,42.9,51,42.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,2017,28,NYG,14,1224,27,13,19,388,1052,...,1933,464,10,44,53,29,54.7,313,39.6,29.4
188,2017,28,WAS,12,1097,41,16,23,388,1045,...,2146,472,13,41,57,29,50.9,307,36.7,60.0
189,2017,30,IND,10,1165,26,13,20,404,1026,...,1929,487,15,44,51,27,52.9,324,44.7,50.0
190,2017,31,CLE,6,1132,34,7,13,410,1026,...,1566,467,14,48,46,31,67.4,315,40.6,50.0


In [26]:
defenses['Opp'] = defenses['NAME']

In [27]:
#change year 
clean_tb['Year'] = clean_tb['Year'].astype(int)

#  Combine the Player Stats with the defenses they played.

In [28]:
#combine the dataframes
tb2 = clean_tb.merge(defenses, on=['Year', 'Opp'], suffixes=('_left','_right'))

In [30]:
tb2

,Rk,Year,Date,G#,Week,Age,Tm,Home,Opp,Cmp,...,RUSH YDS,RUSH ATT,RUSH TD,TD_right,RZ ATT,RZ TD,RZ TD%,1D,3D%,4D%
0,273.0,2017,2017-09-07,1.0,1.0,40.035,NWE,1.0,KAN,16.0,...,1890,443,15,39,51,28,54.9,352,40.1,52.9
1,274.0,2017,2017-09-17,2.0,2.0,40.045,NWE,0.0,NOR,30.0,...,1787,406,11,36,48,25,52.1,308,41.0,33.3
2,275.0,2017,2017-09-24,3.0,3.0,40.052,NWE,1.0,HOU,25.0,...,1747,440,14,48,54,33,61.1,301,35.3,62.5
3,276.0,2017,2017-10-01,4.0,4.0,40.059,NWE,1.0,CAR,32.0,...,1409,353,7,34,44,21,47.7,282,37.9,35.0
4,277.0,2017,2017-10-05,5.0,5.0,40.063,NWE,0.0,TAM,30.0,...,1880,440,18,44,53,29,54.7,327,48.1,22.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,352.0,2021,2022-01-02,16.0,17.0,44.152,TAM,0.0,NYJ,34.0,...,2351,525,29,58,74,45,60.8,401,44.4,61.9
80,354.0,2022,2022-09-11,1.0,1.0,45.039,TAM,0.0,DAL,18.0,...,408,83,1,3,4,2,50.0,59,33.3,50.0
81,355.0,2022,2022-09-18,2.0,2.0,45.046,TAM,0.0,NOR,18.0,...,418,99,2,6,6,2,33.3,51,31.8,0.0
82,356.0,2022,2022-09-25,3.0,3.0,45.053,TAM,1.0,GNB,31.0,...,340,69,1,4,6,3,50.0,49,22.6,66.7


In [29]:
tb2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84 entries, 0 to 83
Data columns (total 66 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rk             84 non-null     float64
 1   Year           84 non-null     int64  
 2   Date           84 non-null     object 
 3   G#             84 non-null     float64
 4   Week           84 non-null     float64
 5   Age            84 non-null     float64
 6   Tm             84 non-null     object 
 7   Home           84 non-null     float64
 8   Opp            84 non-null     object 
 9   Cmp            84 non-null     float64
 10  Att            84 non-null     float64
 11  Cmp%           84 non-null     float64
 12  Yds            84 non-null     float64
 13  TD_left        84 non-null     float64
 14  Int            84 non-null     float64
 15  Rate           84 non-null     float64
 16  Sk             84 non-null     float64
 17  Sk_Yds         84 non-null     float64
 18  Y/A         

# Save the file name as the player's initials_stats and run it in the "Individual_Performance" Notebook

In [ ]:
tb2.to_csv('./tb_stats.csv', index=False)